In [1]:
import pandas as pd
import requests
import os
import sqlite3
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')
db_path

'/Users/paulcolas/code/RonanB400/decp_ml/data/datalab.sqlite'

In [3]:
conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data = pd.read_sql_query(query, conn)
data.columns

Index(['uid', 'id', 'nature', 'acheteur_id', 'acheteur_nom', 'acheteur_siren',
       'titulaire_id', 'titulaire_typeIdentifiant', 'titulaire_nom',
       'titulaire_siren', 'objet', 'montant', 'codeCPV', 'procedure',
       'dureeMois', 'dateNotification', 'datePublicationDonnees', 'formePrix',
       'attributionAvance', 'offresRecues', 'marcheInnovant', 'ccag',
       'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'tauxAvance',
       'origineUE', 'origineFrance', 'lieuExecution_code',
       'lieuExecution_typeCode', 'idAccordCadre', 'source_open_data'],
      dtype='object')

In [4]:
acheteur_siren_df = data['acheteur_siren'].unique()
titulaire_siren_df = data['titulaire_siren'].unique()
print(len(acheteur_siren_df))
print(len(titulaire_siren_df))

10991
62841


In [11]:
data.head()

,uid,id,nature,acheteur_id,acheteur_nom,acheteur_siren,titulaire_id,titulaire_typeIdentifiant,titulaire_nom,titulaire_siren,...,ccag,sousTraitanceDeclaree,typeGroupementOperateurs,tauxAvance,origineUE,origineFrance,lieuExecution_code,lieuExecution_typeCode,idAccordCadre,source_open_data
0,2154005160001320242024-LOT04,20242024-LOT04,Marché,21540051600013,COMMUNE DE BATILLY,215400516,78885470100018,SIRET,ACOMETAL,788854701,...,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,NaN,54980,Code postal,None,data.gouv.fr decp-2024.json
1,243500667002882021M226MO,2021M226MO,Marché,24350066700288,CC VAL D'ILLE-AUBIGNE,243500667,38373211200032,SIRET,UNIVERS,383732112,...,Prestations intellectuelles,1.0,Solidaire,NaN,NaN,NaN,35520,Code postal,None,data.gouv.fr decp-2024.json
2,249710047000472024SS_PRD_TRV,2024SS_PRD_TRV,Marché,24971004700047,COMMUNAUTE DE COMMUNES DE MARIE GALANTE,249710047,43387249600016,SIRET,COTRAM B.T.P.,433872496,...,Travaux,NaN,Solidaire,NaN,NaN,NaN,97112,Code postal,None,data.gouv.fr decp-2024.json
3,6254801990001124-0806-L2,24-0806-L2,Marché,62548019900011,"LA MAISON POUR TOUS, SOCIETE ANONYME COOPERATI...",625480199,55204695502544,SIRET,ENGIE ENERGIE SERVICES,552046955,...,Pas de CCAG,0.0,Pas de groupement,NaN,NaN,NaN,39,Code département,None,data.gouv.fr decp-2024.json
4,20002563300013202424011BCR,202424011BCR,Marché,20002563300013,SI DE RESTAURATION COLLECTIVE,200025633,47698032100238,SIRET,POMONA EPISAVEURS,476980321,...,Fournitures courantes et services,0.0,Pas de groupement,0.0,0.0,0.0,45800,Code postal,None,data.gouv.fr decp-2025-04.json


In [6]:
siren = '215400516'

# Clé API fournie par INSEE
api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"

# URL de l'API Sirene
url = f"https://api.insee.fr/api-sirene/3.11/siren/{siren}"

# Headers avec la clé d'API
headers = {
    "X-INSEE-Api-Key-Integration": api_key
}

# Requête GET
response = requests.get(url, headers=headers).json()

In [7]:
effectif = response['uniteLegale']['trancheEffectifsUniteLegale']
cat_entreprise = response['uniteLegale']['categorieEntreprise']
response['header']['statut']

200

In [16]:
api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"
headers = {"X-INSEE-Api-Key-Integration": api_key}
dico_acheteur_1 = {}

for sir in acheteur_siren_df[:500]:
    url = f"https://api.insee.fr/api-sirene/3.11/siren/{sir}"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            continue

        data = response.json()

        unite_legale = data.get("uniteLegale")
        if not unite_legale:
            continue

        tranche = unite_legale.get("trancheEffectifsUniteLegale")
        if not tranche:
                continue
        categorie = unite_legale.get("categorieEntreprise")
        if not categorie:
            continue

        dico_acheteur_1[sir] = [tranche, categorie]

    except Exception as e:
        continue

In [19]:
dico_acheteur_1

{'215400516': ['12', 'PME'],
 '243500667': ['22', 'PME'],
 '249710047': ['12', 'PME'],
 '625480199': ['22', 'ETI'],
 '200025633': ['12', 'PME'],
 '218502888': ['22', 'PME'],
 '247000011': ['22', 'PME'],
 '213903008': ['32', 'ETI'],
 '214001927': ['32', 'ETI'],
 '216404228': ['31', 'PME'],
 '240900431': ['11', 'PME'],
 '200070092': ['32', 'ETI'],
 '216902593': ['51', 'ETI'],
 '200072734': ['32', 'PME'],
 '218301307': ['32', 'ETI'],
 '216902569': ['42', 'ETI'],
 '219505856': ['51', 'ETI'],
 '228400016': ['51', 'ETI'],
 '315667410': ['32', 'ETI'],
 '221500010': ['42', 'ETI'],
 '249730037': ['21', 'PME'],
 '210601522': ['41', 'ETI'],
 '218708501': ['51', 'ETI'],
 '381016468': ['22', 'PME'],
 '216001743': ['42', 'ETI'],
 '214000465': ['32', 'ETI'],
 '242900769': ['32', 'ETI'],
 '271927212': ['21', 'PME'],
 '860200310': ['22', 'ETI']}

In [20]:
acheteur_siren_df[29]

'860200310'

In [ ]:
api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"
headers = {"X-INSEE-Api-Key-Integration": api_key}
dico_dico = {}

for sir in acheteur_siren_df:
    url = f"https://api.insee.fr/api-sirene/3.11/siren/{sir}"
    response = requests.get(url, headers=headers).json()
    effectif_categorie = [response['uniteLegale']['trancheEffectifsUniteLegale'], response['uniteLegale']['categorieEntreprise']]
    dico_dico[sir] = effectif_categorie

KeyError: 'uniteLegale'

In [8]:
dico_dico = {}

In [9]:
dict_effectif = {
    'NN' : "Unité non-employeuse ou présumée non-employeuse (faute de déclaration reçue)",
    '00' : "0 salarié (n'ayant pas d'effectif au 31/12 mais ayant employé des salariés au cours de l'année de référence)",
    '01' : "1 ou 2 salariés",
    '02' : "3 à 5 salariés",
    '03' : "6 à 9 salariés",
    '11' : "10 à 19 salariés",
    '12' : "20 à 49 salariés",
    '21' : "50 à 99 salariés",
    '22' : "100 à 199 salariés",
    '31' : "200 à 249 salariés",
    '32' : "250 à 499 salariés",
    '41' : "500 à 999 salariés",
    '42' : "1 000 à 1 999 salariés",
    '51' : "2 000 à 4 999 salariés",
    '52' : "5 000 à 9 999 salariés",
    '53' : "10 000 salariés et plus",
    'null' : 'donnée manquante ou "sans objet"'
}

In [10]:
effectif = response['uniteLegale']['trancheEffectifsUniteLegale']
cat_entreprise = response['uniteLegale']['categorieEntreprise']

In [21]:
siren = '215400516'

# Clé API fournie par INSEE
api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"

# URL de l'API Sirene
url = 'https://api.insee.fr/api-sirene/3.11/siren/249710047'

# Headers avec la clé d'API
headers = {
    "X-INSEE-Api-Key-Integration": api_key
}

# Requête GET
response = requests.get(url, headers=headers).json()

In [22]:
response

{'header': {'statut': 200, 'message': 'OK'},
 'uniteLegale': {'siren': '249710047',
  'statutDiffusionUniteLegale': 'O',
  'dateCreationUniteLegale': '1994-01-18',
  'sigleUniteLegale': None,
  'sexeUniteLegale': None,
  'prenom1UniteLegale': None,
  'prenom2UniteLegale': None,
  'prenom3UniteLegale': None,
  'prenom4UniteLegale': None,
  'prenomUsuelUniteLegale': None,
  'pseudonymeUniteLegale': None,
  'identifiantAssociationUniteLegale': None,
  'trancheEffectifsUniteLegale': '12',
  'anneeEffectifsUniteLegale': '2022',
  'dateDernierTraitementUniteLegale': '2025-01-15T05:47:26.431',
  'nombrePeriodesUniteLegale': 5,
  'categorieEntreprise': 'PME',
  'anneeCategorieEntreprise': '2022',
  'periodesUniteLegale': [{'dateFin': None,
    'dateDebut': '2019-01-01',
    'etatAdministratifUniteLegale': 'A',
    'changementEtatAdministratifUniteLegale': False,
    'nomUniteLegale': None,
    'changementNomUniteLegale': False,
    'nomUsageUniteLegale': None,
    'changementNomUsageUniteLegal

In [ ]:
'https://api.insee.fr/api-sirene/3.11/siren?q={249710047,215400516}'